code from sklearn "Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation" example

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
!pip install  -U pbr
!pip install -U lda 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt

import lda
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import seaborn as sns
import os
import glob
import random
print(os.listdir("../input"))
import seaborn as sns


# Any results you write to the current directory are saved as output.
import nltk
from nltk import ngrams

import spacy
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn.decomposition import  PCA,NMF, LatentDirichletAllocation
from dask.distributed import Client
import joblib
#nlp_fr = spacy.load("fr_core_news_sm")

In [ ]:

try:
#     client = Client('127.0.0.1:8786',timeout=5,set_as_default=True)
    client =Client(timeout=5,set_as_default=True,processes=False,n_workers=3,memory_limit='auto', silence_logs='error')
except ValueError as e:
    client =Client(timeout=5,set_as_default=True,processes=False, silence_logs='error')
except OSError as e:
#     print(e)
    client =Client(set_as_default=True,n_workers=2,threads_per_worker=4,dashboard_address=None,processes=False, silence_logs='error')
client

In [ ]:
import spacy.lang.fr
from stop_words import get_stop_words
stopwords_fr_set=set(nltk.corpus.stopwords.words('french'))
stopwords_fr_set.update(get_stop_words('fr'))
stopwords_fr_set.update(spacy.lang.fr.stop_words.STOP_WORDS)
stopwords_fr_set.update(["c'est","j'ai","n'est","n'ait","ca","ça","sais","jamais","chose","ex","'quelqu'",'quelqu',"quelqu'","faut","faudrait"])
stopwords_fr_set=list(stopwords_fr_set)


In [ ]:
n_samples = 1024*1024
n_features = 3000
n_components = 160
n_top_words = 15
ngram_range=(1,3)
min_df=16
max_df=0.25

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,
                                   max_features=n_features,
                                   stop_words=stopwords_fr_set,
                                   ngram_range=ngram_range
                                   )
tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,
                                max_features=n_features,
                                stop_words=stopwords_fr_set,
                                ngram_range=ngram_range)
tfidf_transformer=TfidfTransformer()

In [ ]:
texts=[]
texts_byfiles=dict()
filelist=glob.glob("../input/**/*.csv*", recursive=True)
random.shuffle(filelist)
for f in filelist:
    print (f)
    df=pd.read_csv(f,low_memory=False)
    dftext=[]
    texts_byfiles["f"]=dftext
    for n,s in df.items():
        for e in s:
            if isinstance(e,str):
                if len(e.split())>2 :
                    dftext.append(e)
    texts+=dftext

texts=list(set(texts))

random.shuffle(texts)
texts_all=texts.copy()

In [ ]:
len(texts)

In [ ]:
if n_samples<len(texts):
    texts=random.sample(texts,n_samples)

In [ ]:
with joblib.parallel_backend('dask'):
    tf = tf_vectorizer.fit_transform(texts)
tf

In [ ]:
tf_vectorizer.stop_words

In [ ]:
with joblib.parallel_backend('dask'):
    tfidf=tfidf_transformer.fit_transform(tf)
tfidf

In [ ]:
tf_vectorizer

In [ ]:
tf_vectorizer.stop_words

In [ ]:

rr = dict(zip(tf_vectorizer.get_feature_names(),  tfidf_transformer.idf_))

token_weight = pd.DataFrame.from_dict(rr, orient='index').reset_index()
del rr

token_weight.columns=('token','weight')
token_weight = token_weight.sort_values(by='weight', ascending=False)
token_weight.reset_index(drop=True,inplace=True) 
token_weight["freq"]=1/token_weight.weight

sns.barplot(x='token', y='weight', data=token_weight.iloc[:60], )            
plt.title("Inverse Document Frequency(idf) per token")
fig=plt.gcf()
fig.set_size_inches(25,15)
ax=fig.axes[0]
ax.tick_params(axis='x',labelrotation=90 )
plt.show()


token_weight

In [ ]:
sns.barplot(x='token', y='freq', data=token_weight.iloc[:60], )            
plt.title("Inverse Document Frequency(idf) per token")
fig=plt.gcf()
fig.set_size_inches(25,15)
ax=fig.axes[0]
ax.tick_params(axis='x',labelrotation=90 )
plt.show()

In [ ]:
token_weight.tail(20)

In [ ]:
token_weight.head(20)

In [ ]:

with joblib.parallel_backend('dask'):
    nmf = NMF(n_components=n_components,
              alpha=.1, 
              l1_ratio=.5,
    #           tol=1e-3,
              verbose=True,
              max_iter=20
             ).fit(tfidf)
nmf

In [ ]:

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
def get_top_words_list(model, feature_names, n_top_words):
    topwords=[]
    for topic_idx, topic in enumerate(model.components_):
       
        topwords.append([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
    return topwords



In [ ]:
tfidf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
nmf_top_words_list=get_top_words_list(nmf, tfidf_feature_names, n_top_words)

In [ ]:
with joblib.parallel_backend('dask'):
    tfidf_nmf=nmf.transform(tfidf)
tfidf_nmf

In [ ]:
ind_text=np.random.choice(n_samples,5)

for t,topics in zip([texts[i] for i in ind_text ],
                    
        tfidf_nmf[ind_text]):
    print(t)
    top_topics=np.argsort(topics)[-3:]
    for n in top_topics:
        print(f"topic {n}: {', '.join(nmf_top_words_list[n][:6])}")
    print()
    


In [ ]:
with joblib.parallel_backend('dask'):
    pca_nmf=sklearn.decomposition.KernelPCA(n_components=2,eigen_solver= "arpack")
    pca_nmf.fit(tfidf_nmf[np.random.choice(n_samples,9000)])
    pca_nmf.fit(tfidf_nmf[np.random.choice(n_samples,9000)])
    tfidf_nmf_pca=pca_nmf.fit_transform(tfidf_nmf[np.random.choice(n_samples,9000)])

sns.scatterplot(x="pca1",y="pca2",data=pd.DataFrame(tfidf_nmf_pca,columns=["pca1","pca2"]))


In [ ]:
with joblib.parallel_backend('dask'):
    lda = lda.LDA(n_topics=n_components,
                                    n_iter =5
                                   )

    tfidf_lda=lda.fit_transform(tf)


print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
lda_top_words_list=get_top_words_list(lda, tf_feature_names, n_top_words)
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
lda_best_topics=np.argsort(tfidf_lda.mean(axis=0))[-10:]
for n in range(6):
    print(f"topic {lda_best_topics[-n]}: {', '.join(lda_top_words_list[lda_best_topics[-n]][:15])}")
    


In [ ]:
topic_texts=np.argsort(tfidf_lda,axis=0)[:300]
for t in range(tfidf_lda.shape[1]):
    for i in range(6):
        nmax=topic_texts[-i,t]
        print(tfidf_lda[nmax,t])
        print(texts[nmax][:300])    
    print(f"topic {t}: {', '.join(lda_top_words_list[t][:15])}")
    print("***")

In [ ]:
ind_text=np.random.choice(n_samples,5)

for t,topics in zip([texts[i] for i in ind_text ],
                    
        tfidf_lda[ind_text]):
    print(t)
    top_topics=np.argsort(topics)[-3:]
    for n in top_topics:
        print(f"topic {n}: {', '.join(nmf_top_words_list[n][:6])}")
    print()

In [ ]:


print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer_n.get_feature_names()
lda_top_words_list=get_top_words_list(lda, tf_feature_names, n_top_words)
print_top_words(lda, tf_feature_names, n_top_words)